Resize images to prepare datasets before pushing them to the repo

In [ ]:
%matplotlib inline

# General imports
import matplotlib.pyplot as plt
import pathlib
import skimage
import skimage.transform
import skimage.viewer
import pandas as pd
import numpy as np
import time
import keras.utils.np_utils
from tqdm import tqdm

# Setup to show interactive jupyter widgets
from IPython.display import Image, display
from ipywidgets import interact, fixed
import ipywidgets as widgets
def imgplotList(i,data):
    plt.figure(figsize=(10,10))
    plt.imshow(data[i],interpolation="nearest")
    plt.show()

In [ ]:
# Define where datasets are located
dataset_directory = pathlib.Path("..")/"datasets"/"final"

# Define which datasets we should consider.
# Each dataset is a directory withing dataset_directory
# and must contain three subdirectories: (c0, c1, c2) for (rock, paper, scissors).
dnames = ["testing","D1","D2","D3","D4","D5","D6"]


# Now check the data
ddirs=[dataset_directory/dn for dn in dnames] # directories of the dataset
cdirs={}
for ddir in ddirs:
    cdirs.update({ddir/"c0":0,
                  ddir/"c1":1,
                  ddir/"c2":2})
names = ["rock", "paper", "scissors"]
for cdir,cdir_class in cdirs.items():
    assert(cdir.exists())
    print("Found directory {} containing class {}".format(cdir,names[cdir_class]))

In [ ]:
imagesize = 500

import warnings

for cdir,cn in tqdm(list(cdirs.items())):
    for f in list(cdir.glob("*")):
        try:
            im=skimage.io.imread(f)
        except (OSError, ValueError) as e:
            warnings.warn("ignoring {} due to exception {}".format(f,str(e)))
            continue
            
        h,w=im.shape[0:2] # height, width
        sz=min(h,w)
        im=im[(h//2-sz//2):(h//2+sz//2),(w//2-sz//2):(w//2+sz//2),:] # defines the central square        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            im=skimage.img_as_ubyte(skimage.transform.resize(im,(imagesize,imagesize))) # resize it to 500x500, whatever the original resolution
            
        destdir = f.parent.parent.with_suffix(".s")
        destdir.mkdir(exist_ok = True)
        destdir = destdir/f.parent.name
        destdir.mkdir(exist_ok = True)
        skimage.io.imsave(str((destdir/f.name).with_suffix(".jpg")),
                          im, quality=90)
        
print("Done")